# Basic Recipe: Static Analysis of a Simple Reform

This is the recipe you should follow first.
Mastering this recipe is a prerequisite for all the other recipes in this cookbook.

## Imports

In [6]:
# Install conda and taxcalc if in Google Colab.
import sys
if 'google.colab' in sys.modules and 'taxcalc' not in sys.modules:
    !wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
    !bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
    # Append path to be able to run packages installed with conda
    # This must correspond to the conda Python version, which may differ from
    # the base Colab Python installation.
    sys.path.append('/usr/local/lib/python3.7/site-packages')
    # Install PSL packages from Anaconda
    !yes | conda install -c PSLmodels taxcalc behresp

In [7]:
import taxcalc as tc
import pandas as pd
from bokeh.io import show, output_notebook

## Setup

Use publicly-available CPS input file.

````{note}
If you have access to the restricted-use IRS-SOI PUF-based input file
and you have that file (named `puf.csv`) located in the directory
where this script is located, as well as the weights and adjustment ratios associated with `puf.csv`, then you can substitute `recs = tc.Records.cps_constructor()` with the following:

``` python
puf_data = 'puf.csv'
puf_weights = 'puf_weights_file.csv'
puf_ratios = 'puf_ratios_file.csv'
start_year = YEAR

recs = tc.Records(data=puf_data, weights=puf_weights, adjust_ratios=puf_ratios, start_year=start_year)
```
````

In [8]:
recs = tc.Records.cps_constructor()

Specify Calculator object for static analysis of current-law policy.

In [9]:
pol = tc.Policy()
calc1 = tc.Calculator(policy=pol, records=recs)

NOTE: calc1 now contains a PRIVATE COPY of pol and a PRIVATE COPY of recs,
so we can continue to use pol and recs in this script without any
concern about side effects from Calculator method calls on calc1.

In [10]:
CYR = 2020

Calculate aggregate current-law income tax liabilities for CYR.

In [11]:
calc1.advance_to_year(CYR)
calc1.calc_all()
itax_rev1 = calc1.weighted_total('iitax')

Read JSON reform file and use (the default) static analysis assumptions.

In [12]:
reform_filename = '_static/reformA.json'
params = tc.Calculator.read_json_param_objects(reform_filename, None)

Specify Calculator object for static analysis of reform policy.

In [13]:
pol.implement_reform(params['policy'])
calc2 = tc.Calculator(policy=pol, records=recs)

## Calculate

Calculate reform income tax liabilities for CYR.

In [14]:
calc2.advance_to_year(CYR)
calc2.calc_all()
itax_rev2 = calc2.weighted_total('iitax')

## Results

Print total revenue estimates for 2018.

*Estimates in billons of dollars rounded to nearest hundredth of a billion.*

In [15]:
print('{}_CLP_itax_rev($B)= {:.3f}'.format(CYR, itax_rev1 * 1e-9))
print('{}_REF_itax_rev($B)= {:.3f}'.format(CYR, itax_rev2 * 1e-9))

2020_CLP_itax_rev($B)= 1323.792
2020_REF_itax_rev($B)= 1332.149


Generate several other standard results tables.

In [16]:
# Aggregate diagnostic tables for CYR.
clp_diagnostic_table = calc1.diagnostic_table(1)
ref_diagnostic_table = calc2.diagnostic_table(1)

# Income-tax distribution for CYR with CLP and REF results side-by-side.
dist_table1, dist_table2 = calc1.distribution_tables(calc2, 'weighted_deciles')
assert isinstance(dist_table1, pd.DataFrame)
assert isinstance(dist_table2, pd.DataFrame)
dist_extract = pd.DataFrame()
dist_extract['funits(#m)'] = dist_table1['count']
dist_extract['itax1($b)'] = dist_table1['iitax']
dist_extract['itax2($b)'] = dist_table2['iitax']
dist_extract['aftertax_inc1($b)'] = dist_table1['aftertax_income']
dist_extract['aftertax_inc2($b)'] = dist_table2['aftertax_income']

# Income-tax difference table by expanded-income decile for CYR.
diff_table = calc1.difference_table(calc2, 'weighted_deciles', 'iitax')
assert isinstance(diff_table, pd.DataFrame)
diff_extract = pd.DataFrame()
dif_colnames = ['count', 'tot_change', 'mean', 'pc_aftertaxinc']
ext_colnames = ['funits(#m)', 'agg_diff($b)', 'mean_diff($)', 'aftertaxinc_diff(%)']
for dname, ename in zip(dif_colnames, ext_colnames):
    diff_extract[ename] = diff_table[dname]

## Plotting

Generate a decile graph and display it using Bokeh (will render in Jupyter, not in webpage).

In [17]:
fig = calc1.pch_graph(calc2)
output_notebook()
show(fig)

Loading BokehJS ...

## Print tables

CLP diagnostic table for CYR.

In [18]:
clp_diagnostic_table

,2020
Returns (#m),205.440
AGI ($b),11475.571
Itemizers (#m),31.430
Itemized Deduction ($b),832.863
Standard Deduction Filers (#m),174.010
Standard Deduction ($b),3170.709
Personal Exemption ($b),0.000
Taxable Income ($b),8531.399
Regular Tax ($b),1507.622
AMT Income ($b),10849.663


REF diagnostic table for CYR.

In [19]:
ref_diagnostic_table

,2020
Returns (#m),205.440
AGI ($b),11475.571
Itemizers (#m),31.330
Itemized Deduction ($b),830.006
Standard Deduction Filers (#m),174.110
Standard Deduction ($b),3172.307
Personal Exemption ($b),387.209
Taxable Income ($b),8291.539
Regular Tax ($b),1513.597
AMT Income ($b),10852.372


Extract of CYR distribution tables by baseline expanded-income decile.

In [20]:
dist_extract

,funits(#m),itax1($b),itax2($b),aftertax_inc1($b),aftertax_inc2($b)
0-10n,0.114430,0.000000,0.000000,-8.336880,-8.336880
0-10z,8.403193,0.000000,0.000000,0.000000,0.000000
0-10p,12.024905,-0.653935,-0.653935,39.096355,39.096355
10-20,20.546100,-8.109892,-8.423973,254.108172,254.422253
20-30,20.544257,-6.802310,-7.878318,483.595862,484.671870
30-40,20.544358,0.536044,-0.723522,662.241763,663.501329
40-50,20.543396,6.925946,5.287958,826.574357,828.212345
50-60,20.544877,17.458704,15.054502,1026.900590,1029.304792
60-70,20.544650,35.900009,32.407618,1288.397820,1291.890211
70-80,20.544138,75.086248,70.394631,1638.497067,1643.188684


Extract of CYR income-tax difference table by expanded-income decile.

In [21]:
diff_extract

,funits(#m),agg_diff($b),mean_diff($),aftertaxinc_diff(%)
0-10n,0.114430,0.000000e+00,0.000000e+00,0.000000
0-10z,8.403193,0.000000e+00,0.000000e+00,NaN
0-10p,12.024905,1.629896e-22,1.355434e-20,0.000000
10-20,20.546100,-3.140808e-01,-1.528664e+01,0.123601
20-30,20.544257,-1.076008e+00,-5.237513e+01,0.222502
30-40,20.544358,-1.259566e+00,-6.130959e+01,0.190197
40-50,20.543396,-1.637988e+00,-7.973305e+01,0.198166
50-60,20.544877,-2.404202e+00,-1.170220e+02,0.234122
60-70,20.544650,-3.492391e+00,-1.699903e+02,0.271065
70-80,20.544138,-4.691617e+00,-2.283677e+02,0.286337
